In [12]:
import torch
import numpy as np
import mindspore
from mindspore import Tensor, dtype
from mindspore import save_checkpoint
from mindspore import Parameter
model = torch.load("alt_gvt_base.pth", map_location='cpu')


In [19]:
model_keys = []
nums = 0.
for key in model.keys():
    if "num_batches_tracked" not in key:
        model_keys.append(key)

In [20]:
weights = []
prefix = "model."
for key in model_keys:
    name2weight = {}
    if "bn" in key or "norm" in key or "ln" in key:
        if "weight" in key:
            name2weight["name"] = prefix + key.replace(".weight", ".gamma")
        elif "bias" in key:
            name2weight["name"] = prefix + key.replace(".bias", ".beta")
        elif "mean" in key:
            name2weight["name"] = prefix + key.replace("running_mean", "moving_mean")
        elif "var" in key:
            name2weight["name"] = prefix + key.replace("running_var", "moving_variance")
        
        name2weight["data"] = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weights.append(name2weight)
    elif "qkv" in key:
        key_q = prefix +  key.replace("qkv", "q")
        key_k = prefix +  key.replace("qkv", "k")
        key_v = prefix + key.replace("qkv", "v")
        shape = model[key].shape[0]//3
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weight_q = weight[:shape]
        weight_k = weight[shape:shape*2]
        weight_v = weight[shape*2:]
        weights.append({"name":key_q, "data": weight_q})
        weights.append({"name":key_k, "data": weight_k})
        weights.append({"name":key_v, "data": weight_v})
    elif "kv" in key:
        key_k = prefix +  key.replace("kv", "k")
        key_v = prefix +  key.replace("kv", "v")
        shape = model[key].shape[0]//2
        weight = Parameter(Tensor(model[key].numpy(), dtype.float32),requires_grad=True) 
        weight_k = weight[:shape]
        weight_v = weight[shape:shape*2]
        weights.append({"name":key_k, "data": weight_k})
        weights.append({"name":key_v, "data": weight_v})
    else:
        dd = dtype.float32
        if "relative_position_index" in key:
            dd = dtype.int32
        weight = Parameter(Tensor(model[key].numpy(), dd),requires_grad=True)
        key = prefix + key
        weights.append({"name": key, "data": weight})
    print(key)

model.patch_embeds.0.proj.weight
model.patch_embeds.0.proj.bias
patch_embeds.0.norm.weight
patch_embeds.0.norm.bias
model.patch_embeds.1.proj.weight
model.patch_embeds.1.proj.bias
patch_embeds.1.norm.weight
patch_embeds.1.norm.bias
model.patch_embeds.2.proj.weight
model.patch_embeds.2.proj.bias
patch_embeds.2.norm.weight
patch_embeds.2.norm.bias
model.patch_embeds.3.proj.weight
model.patch_embeds.3.proj.bias
patch_embeds.3.norm.weight
patch_embeds.3.norm.bias
norm.weight
norm.bias
model.head.weight
model.head.bias
model.pos_block.0.proj.0.weight
model.pos_block.0.proj.0.bias
model.pos_block.1.proj.0.weight
model.pos_block.1.proj.0.bias
model.pos_block.2.proj.0.weight
model.pos_block.2.proj.0.bias
model.pos_block.3.proj.0.weight
model.pos_block.3.proj.0.bias
blocks.0.0.norm1.weight
blocks.0.0.norm1.bias
blocks.0.0.norm2.weight
blocks.0.0.norm2.bias
model.blocks.0.0.mlp.fc1.weight
model.blocks.0.0.mlp.fc1.bias
model.blocks.0.0.mlp.fc2.weight
model.blocks.0.0.mlp.fc2.bias
blocks.0.0.attn.

blocks.2.14.attn.qkv.weight
blocks.2.14.attn.qkv.bias
model.blocks.2.14.attn.proj.weight
model.blocks.2.14.attn.proj.bias
blocks.2.15.norm1.weight
blocks.2.15.norm1.bias
blocks.2.15.norm2.weight
blocks.2.15.norm2.bias
model.blocks.2.15.mlp.fc1.weight
model.blocks.2.15.mlp.fc1.bias
model.blocks.2.15.mlp.fc2.weight
model.blocks.2.15.mlp.fc2.bias
model.blocks.2.15.attn.q.weight
model.blocks.2.15.attn.q.bias
blocks.2.15.attn.kv.weight
blocks.2.15.attn.kv.bias
model.blocks.2.15.attn.proj.weight
model.blocks.2.15.attn.proj.bias
model.blocks.2.15.attn.sr.weight
model.blocks.2.15.attn.sr.bias
blocks.2.15.attn.norm.weight
blocks.2.15.attn.norm.bias
blocks.2.16.norm1.weight
blocks.2.16.norm1.bias
blocks.2.16.norm2.weight
blocks.2.16.norm2.bias
model.blocks.2.16.mlp.fc1.weight
model.blocks.2.16.mlp.fc1.bias
model.blocks.2.16.mlp.fc2.weight
model.blocks.2.16.mlp.fc2.bias
blocks.2.16.attn.qkv.weight
blocks.2.16.attn.qkv.bias
model.blocks.2.16.attn.proj.weight
model.blocks.2.16.attn.proj.bias
blocks

In [21]:
weights_out = []
for weight in weights:
    weight_out = {}
    name = weight["name"]
    weight_out["name"] = name
    weight_out["data"] = weight["data"]
    weights_out.append(weight_out)
    print(weight_out["name"], weight_out['data'].shape)

model.patch_embeds.0.proj.weight (96, 3, 4, 4)
model.patch_embeds.0.proj.bias (96,)
model.patch_embeds.0.norm.gamma (96,)
model.patch_embeds.0.norm.beta (96,)
model.patch_embeds.1.proj.weight (192, 96, 2, 2)
model.patch_embeds.1.proj.bias (192,)
model.patch_embeds.1.norm.gamma (192,)
model.patch_embeds.1.norm.beta (192,)
model.patch_embeds.2.proj.weight (384, 192, 2, 2)
model.patch_embeds.2.proj.bias (384,)
model.patch_embeds.2.norm.gamma (384,)
model.patch_embeds.2.norm.beta (384,)
model.patch_embeds.3.proj.weight (768, 384, 2, 2)
model.patch_embeds.3.proj.bias (768,)
model.patch_embeds.3.norm.gamma (768,)
model.patch_embeds.3.norm.beta (768,)
model.norm.gamma (768,)
model.norm.beta (768,)
model.head.weight (1000, 768)
model.head.bias (1000,)
model.pos_block.0.proj.0.weight (96, 1, 3, 3)
model.pos_block.0.proj.0.bias (96,)
model.pos_block.1.proj.0.weight (192, 1, 3, 3)
model.pos_block.1.proj.0.bias (192,)
model.pos_block.2.proj.0.weight (384, 1, 3, 3)
model.pos_block.2.proj.0.bias (38

In [22]:
save_checkpoint(weights_out, "alt_gvt_base.ckpt")

In [23]:
len(weights_out)

456